This notebook is aiming at preparing training data ready for comprehend custom classifier "multi-label" model using Vanilla strategy (with minimum required preprocessing)

Prerequisite:

1) All csv files generated by prior notebooks

Output:

1) A csv file containing prepared training data ready for comprehend

In [1]:
# Import required libraries
import pandas as pd
import numpy as np

In [2]:
# Load prior prepared extracted text files
textract_df = pd.read_csv("Textract_output.csv")
textract_df.rename(columns={'file_name' : 'filename'}, inplace=True)
textract_df

filename  page  \
0                 1003700000000530903_11159719.pdf     1   
1                 1003700000000530903_11159719.pdf     2   
2                 1003700000000530903_11159719.pdf     3   
3                 1003700000000530903_11159719.pdf     4   
4                 1003700000000530903_11159719.pdf     5   
...                                            ...   ...   
840163  OCR_33049_1003700000000697103_11103804.pdf    43   
840164  OCR_33049_1003700000000697103_11103804.pdf    44   
840165  OCR_33049_1003700000000697103_11103804.pdf    45   
840166  OCR_33049_1003700000000697103_11103804.pdf    46   
840167  OCR_33049_1003700000000697103_11103804.pdf    47   

                                                  content  
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...  
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...  
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...  
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...  
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...  
...                                                   ...  
840163  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  
840164  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840165  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840166  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840167  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  

[840168 rows x 3 columns]

In [3]:
# Change filename to same format 
textract_df['filename'] = textract_df['filename'].str.split('_').str[-2] + '_' + textract_df['filename'].str.split('_').str[-1]

In [4]:
# Load prior prepared reformat label file
label_df = pd.read_csv("batch3_label_revised.csv")

In [5]:
# Check common file
common_files = set(textract_df.filename).intersection(label_df.filename)
len(common_files)

2378

In [6]:
# filter out uncommon file
textract_df = textract_df[textract_df.filename.isin(common_files)]
textract_df

filename  page  \
0       1003700000000530903_11159719.pdf     1   
1       1003700000000530903_11159719.pdf     2   
2       1003700000000530903_11159719.pdf     3   
3       1003700000000530903_11159719.pdf     4   
4       1003700000000530903_11159719.pdf     5   
...                                  ...   ...   
840163  1003700000000697103_11103804.pdf    43   
840164  1003700000000697103_11103804.pdf    44   
840165  1003700000000697103_11103804.pdf    45   
840166  1003700000000697103_11103804.pdf    46   
840167  1003700000000697103_11103804.pdf    47   

                                                  content  
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...  
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...  
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...  
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...  
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...  
...                                                   ...  
840163  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  
840164  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840165  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840166  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840167  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  

[838573 rows x 3 columns]

In [7]:
# merge text files with labels
merged_file = pd.merge(textract_df, label_df, how = "left", on = ["filename", "page"])
merged_file

filename  page  \
0       1003700000000530903_11159719.pdf     1   
1       1003700000000530903_11159719.pdf     2   
2       1003700000000530903_11159719.pdf     3   
3       1003700000000530903_11159719.pdf     4   
4       1003700000000530903_11159719.pdf     5   
...                                  ...   ...   
859387  1003700000000697103_11103804.pdf    43   
859388  1003700000000697103_11103804.pdf    44   
859389  1003700000000697103_11103804.pdf    45   
859390  1003700000000697103_11103804.pdf    46   
859391  1003700000000697103_11103804.pdf    47   

                                                  content      class  \
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...        NaN   
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...        NaN   
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...        NaN   
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...        NaN   
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...        NaN   
...                                                   ...        ...   
859387  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  radiology   
859388  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   
859389  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   
859390  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   
859391  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   

        filename_cut  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  
...              ...  
859387      697103.0  
859388      697103.0  
859389      697103.0  
859390      697103.0  
859391      697103.0  

[859392 rows x 5 columns]

In [8]:
# Fill unlabelled data to "others" class
merged_file['class'] = merged_file['class'].fillna('others')
merged_file

filename  page  \
0       1003700000000530903_11159719.pdf     1   
1       1003700000000530903_11159719.pdf     2   
2       1003700000000530903_11159719.pdf     3   
3       1003700000000530903_11159719.pdf     4   
4       1003700000000530903_11159719.pdf     5   
...                                  ...   ...   
859387  1003700000000697103_11103804.pdf    43   
859388  1003700000000697103_11103804.pdf    44   
859389  1003700000000697103_11103804.pdf    45   
859390  1003700000000697103_11103804.pdf    46   
859391  1003700000000697103_11103804.pdf    47   

                                                  content      class  \
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...     others   
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...     others   
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...     others   
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...     others   
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...     others   
...                                                   ...        ...   
859387  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  radiology   
859388  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   
859389  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   
859390  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   
859391  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   

        filename_cut  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  
...              ...  
859387      697103.0  
859388      697103.0  
859389      697103.0  
859390      697103.0  
859391      697103.0  

[859392 rows x 5 columns]

In [9]:
# Prepare data ready for "multi-label" mode
merged_file = merged_file[['content', 'class']].drop_duplicates().groupby('content')['class'].apply('|'.join).reset_index()

In [ ]:
# To prepare for training data, only select "class" and "content" columns and keey it in right sequence
merged_file = merged_file[['class', 'content']]

In [ ]:
# Save prepared training data to local
merged_file.to_csv("batch3_combined_multi_label.csv", index=False)

###### Optional: Add prior records to training data

This optional field applies to the case when there were training data from prior training datasets. The training data should be ready for comprehend training use already.

In [10]:
# Load prior training data
batch2_file = pd.read_csv('training_data_multi_label_phase3_data.csv', names=['class', 'content'])
batch2_file

class                                            content
0                  others  ! 5SOUTH NASSAUINMUNITIES HOSPITALMedical Reco...
1       discharge_summary  ! u 191218905785004Weiss Memorial HospitalPati...
2                  others  !(use first)SOUTH NASSAU COMMUNITIES HOSPITALO...
3               radiology  !.ast Name: TURNERFirst Name: AURTHURPatient I...
4                  others  !06$69608UPS Restrictions00000200 NOSLS LISIA ...
...                   ...                                                ...
677686          radiology  Ürders and Results (continued)SHOULDER 3 VIEWS...
677687             others  ç !Swedish Covenar.t Hospital LIVEPatient Orde...
677688             others  é 85Jun 02 2018 17:14:46 Via Fax9797765333AT&T...
677689             others  édication AlhistrationbrdAdministraçico Dhon12...
677690               labs  ₹10408509197RR0R11CUMULATIVE REPORTPage:220037...

[677691 rows x 2 columns]

In [12]:
# Merge datasets together
merged_df = pd.concat([batch2_file, merged_file], axis=0, ignore_index=True)
merged_df

class                                            content
0                   others  ! 5SOUTH NASSAUINMUNITIES HOSPITALMedical Reco...
1        discharge_summary  ! u 191218905785004Weiss Memorial HospitalPati...
2                   others  !(use first)SOUTH NASSAU COMMUNITIES HOSPITALO...
3                radiology  !.ast Name: TURNERFirst Name: AURTHURPatient I...
4                   others  !06$69608UPS Restrictions00000200 NOSLS LISIA ...
...                    ...                                                ...
1509808             others  © 2000-2021 The StayWell Company, LLC. All rig...
1509809             others  © 2016, Elsevier/Gold Standard. (2015-11-18 15...
1509810             others  àPage: 173PIERCE, HENRY CHARLESFac: St. Joseph...
1509811             others  àPage: 323PIERCE, HENRY CHARLESFac: St. Joseph...
1509812             others  ⑇~1356072~51260774~I&O_RPT~##ENHMC VRF ReportL...

[1509813 rows x 2 columns]

In [13]:
# Check distribution
merged_df['class'].value_counts()

others                                                               892764
flowsheets                                                           184756
progress_notes                                                       103541
mar                                                                   81604
labs                                                                  71156
                                                                      ...  
discharge_summary|history_physical|procedures                             1
discharge_summary|admission_order|emergency_department                    1
admission_reason|physician_query|discharge_summary|progress_notes         1
progress_notes|procedures|mar                                             1
history_physical|procedures|consult                                       1
Name: class, Length: 965, dtype: int64

##### Limitation

According to comprehend service quotas(https://docs.aws.amazon.com/comprehend/latest/dg/guidelines-and-limits.html): Maximum number of lines, one document per line (for all files in request) is 1,000,000.

So if the training data has more than 1,000,000 records, we should remove some files.

Since in this training set, "others" class is very dominant but containing less information, we will randomly remove 550,000 records from "others" class.

In [14]:
## Drop OTHERS records

frac_num = 550000
merged_df_dropped = merged_df.drop(merged_df[merged_df['class']=='others'].sample(frac_num).index)

In [15]:
# Check distribution
merged_df_dropped['class'].value_counts()

others                                                               342764
flowsheets                                                           184756
progress_notes                                                       103541
mar                                                                   81604
labs                                                                  71156
                                                                      ...  
discharge_summary|history_physical|procedures                             1
discharge_summary|admission_order|emergency_department                    1
admission_reason|physician_query|discharge_summary|progress_notes         1
progress_notes|procedures|mar                                             1
history_physical|procedures|consult                                       1
Name: class, Length: 965, dtype: int64

In [ ]:
# Store data locally
merged_df_dropped.to_csv('training_data_multi_label_batch123.csv', index=False, header=False)

###### Optional: Add keyword records

This optional field applies to the case when we want to add keyword records to training data

In [16]:
# Load keyword file
keywords_df = pd.read_csv('keywords_to_training.csv')

In [17]:
# Merge keyword records to training file
merged_df_dropped_keywords = pd.concat([merged_df_dropped, keywords_df], axis=0, ignore_index=True)

In [18]:
# Shuffle records
merged_df_dropped_keywords_shuffled = merged_df_dropped_keywords.sample(frac=1)

In [ ]:
# Save file to local
merged_df_dropped_keywords_shuffled.to_csv('training_data_multi_label_batch123_with_keywords.csv', index=False, header=False)